In [1]:
import random
import time
import re

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains

from pyvirtualdisplay import Display
from bs4 import BeautifulSoup
from PIL import Image
import requests
import cv2

In [2]:
def get_Proxy():
#     url = 'http://192.168.2.120:5000'
    url = 'http://120.27.216.150:5000'
    headers = {
        'Connection': 'close',
    }
    proxy = requests.get(url, headers=headers, auth=('admin', 'zd123456'), timeout=30).text[0:-6]
    return proxy

# 车300 滑块验证码

In [12]:
class Yanzhengma():
    def __init__(self):
        self.chrome_opts = webdriver.ChromeOptions()
#         self.display = Display(visible=False, size=(800, 800))
#         self.display.start()  
#         self.chrome_opts.add_argument('--headless')
        self.ua = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.111 Safari/537.36'
        proxy = get_Proxy()
        print('使用代理IP:  ', proxy)
        self.chrome_opts.add_argument('--proxy-server=' + proxy)
        self.chrome_opts.add_argument('--no-sandbox')
        self.chrome_opts.add_argument('--disable-gpu')  # 规避bug
        self.chrome_opts.add_experimental_option('excludeSwitches', ['enable-automation'])
        self.chrome_opts.add_argument('--disable-dev-shm-usage')
        self.chrome_opts.add_argument('--user-agent=' + self.ua)
        self.driver = webdriver.Chrome(options=self.chrome_opts)
        self.driver.maximize_window()
        self.driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
            "source": """
            Object.defineProperty(navigator, 'webdriver', {
              get: () => undefined
            })
          """
        })
        self.wait = WebDriverWait(self.driver, 10)
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36',
        }
        
    def __del__(self):
#         self.display.stop()
        self.driver.quit()
        
        
    def get_pic(self, url, i):
        self.driver.get(url)
        test_image = self.wait.until(EC.presence_of_element_located((By.ID, 'dx_captcha_basic_bg_1')))
        self.driver.save_screenshot('che300.png')
        
        soup = BeautifulSoup(self.driver.page_source, 'html5lib')
        
        # 下载保存缺口小图片        
        try:
            img_url = soup.select('div#dx_captcha_basic_sub-slider_1 img')[0].get('src')
#             print(img_url)
            with open('slider.webp', 'wb') as f:
                f.write(requests.get(img_url, headers=self.headers).content)
            slider = Image.open('slider.webp')
            slider.load()
            slider.save('slider_{:03d}.png'.format(i))
        except Exception as e:
            print(repr(e))
            
#         # 在保存的页面截图中 截取验证码图片
        left, top, right, bottom = test_image.location['x'], test_image.location['y'], \
                                    test_image.location['x'] + test_image.size['width'],\
                                    test_image.location['y'] + test_image.size['height']
        pic = Image.open('che300.png')
        yanzheng_img = pic.crop((left, top, right, bottom))
        yanzheng_img.save('验证码_{:03d}.png'.format(i))
        
    def get_offset(self, i):
        '''
        使用cv2获取图片缺口偏移量
        '''
        target_rgb = cv2.imread('验证码_{:03d}.png'.format(i))
        template_gray = cv2.imread('slider_{:03d}.png'.format(i), 0)
        target_gray = cv2.cvtColor(target_rgb, cv2.COLOR_BGR2GRAY)
        ret, threshed = cv2.threshold(target_gray, 127, 255, cv2.THRESH_BINARY)
        
        # 新加代码
        contours, hierarchy = cv2.findContours(threshed, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
#         rett, threshedt = cv2.threshold(template_rgb, 127, 255, cv2.THRESH_BINARY)
        cnt = contours[0]
    
    
        res = cv2.matchTemplate(threshed, template_gray, cv2.TM_CCOEFF_NORMED)
        value = cv2.minMaxLoc(res)
        print(value)
        if abs(value[0]) > abs(value[1]):
            distance = value[2][0]
            print('缺口偏移量：', distance)
        else:
            distance = value[3][0]
            print('缺口偏移量：', distance)
            
        return distance
    
    def get_track(self, distance):
        '''
        根据偏移量获取移动轨迹
        '''
        tracks = []    # 移动轨迹
        # 当前位移
        current = 0
        # 减速阈值
        mid = distance * 4 / 5
        # 计算间隔
        t = 0.2
        # 初速度
        v = 0
        
        while current < distance:
            if current < mid:
                a = 10  # 加速度越大，滑动速度越快
            else:
                a = -3  # 加速度为-3
            v0 = v
            # 0.3s时间内的位移
            s = v0 * t + 0.5 * a * (t ** 2)
            # 当前位置
            current += s
            # 添加到轨迹列表
            tracks.append(round(s))
            # 速度已经达到V，该速度作为下次的初速度
            v = v0 + a * t
            
        return tracks
        
    def move(self, track=None):
        ActionChains(self.driver).click_and_hold(self.driver.find_element_by_id("dx_captcha_basic_slider-img-animated-wrap_1")).perform()
        for x in track:
            ActionChains(self.driver).move_by_offset(xoffset=x, yoffset=0).perform()

#         ActionChains(self.driver).move_by_offset(xoffset=track, yoffset=0).perform()
        time.sleep(0.7)
        ActionChains(self.driver).release().perform()
#         print('当前url: ', self.driver.current_url)
        time.sleep(1)
        try:
            step2 = self.driver.find_elements_by_class_name('dx_captcha_em')[0]
            print('滑块拟合成功！\n')
            if 'forbidden' not in self.driver.current_url:
                print(self.driver.page_source)
                return 1
            else:
                self.driver.refresh()
                return 0
        except Exception as e:
            print('失败。。。\n')
#             print('验证码页面：', 'forbidden' in self.driver.current_url)
            return 0
        
        
# 车300 url
if __name__ == "__main__":
    url = 'https://www.che300.com/partner/result.php?prov=22&city=22&brand=30&series=386&model=21359&registerDate=2014-1&mileAge=13.17&intention=0&partnerId=douyin&unit=1&sn=93a15125acc736ab66bb791a1e37ae1a&sld=cd'
#     che300 = Yanzhengma()
    i = 0
    cnt = 0
    while i < 10:
        print(f'第 {i} 次测试')
        try:
            che300 = Yanzhengma()
            time.sleep(0.5)
            che300.get_pic(url, i)
            distance = che300.get_offset(i)
            track = che300.get_track(distance)
            cnt += che300.move(track)
            i += 1
            if cnt == 1:
                break
        except Exception as e:
            print(repr(e))

            
#     print(f'成功的次数：{cnt}')
    print('ok')

第 0 次测试
使用代理IP:   175.4.199.248:27745
TimeoutException('', None, None)
第 0 次测试
使用代理IP:   111.76.175.179:31934
(-0.41147974133491516, 0.5307907462120056, (179, 44), (207, 13))
缺口偏移量： 207
失败。。。

第 1 次测试
使用代理IP:   140.250.91.242:31255
(-0.20586203038692474, 0.19281813502311707, (116, 69), (120, 46))
缺口偏移量： 116
失败。。。

第 2 次测试
使用代理IP:   121.57.166.14:46799
(-0.13315288722515106, 0.05615856125950813, (23, 71), (105, 65))
缺口偏移量： 23
失败。。。

第 3 次测试
使用代理IP:   117.66.140.115:23840
(-0.4544358551502228, 0.2773038446903229, (0, 13), (157, 78))
缺口偏移量： 0
失败。。。

第 4 次测试
使用代理IP:   114.99.12.78:23278
(-0.0691506490111351, 0.10416726768016815, (117, 82), (148, 78))
缺口偏移量： 148
失败。。。

第 5 次测试
使用代理IP:   60.176.20.88:23674
(-0.423415869474411, 0.2972216010093689, (106, 61), (0, 13))
缺口偏移量： 106
失败。。。

第 6 次测试
使用代理IP:   115.151.215.24:47623
(-0.5809280872344971, 0.40375635027885437, (124, 37), (216, 0))
缺口偏移量： 124
失败。。。

第 7 次测试
使用代理IP:   115.151.146.44:33282
(-0.08761080354452133, 0.08808141201734543, (114, 8

In [9]:
chrome_opts = Options()
# proxy = get_Proxy()
# print('使用代理IP:  ', proxy)
# chrome_opts.add_argument('--proxy-server=' + proxy)
# chrome_opts.add_argument('--disable-images')
chrome_opts.add_argument('--incognito')  #  无痕模式
chrome_opts.add_argument('--start-maximized')
# chrome_opts.add_argument('--no-sandbox')
chrome_opts.add_argument('--user-agent="Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36"')
chrome_opts.add_experimental_option('excludeSwitches', ['enable-automation'])
driver = webdriver.Chrome(options=chrome_opts)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": """
        Object.defineProperty(navigator, 'webdriver', {
            get: () => undefined
        })
    """
})
url = 'https://jiage.autohome.com.cn/price/carlist/p-40624-0-0-0-0-0-10-0'
headers = {
            'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36',
        }
res = driver.get(url)
time.sleep(5)
html = driver.page_source
# print(driver.get_cookies())
print(html)

<html lang="en"><head>
  <meta charset="UTF-8">
  <meta http-equiv="X-UA-Compatible" content="IE=edge,chrome=1">
  <meta name="renderer" content="webkit">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <meta http-equiv="X-UA-Compatible" content="ie=edge">
  <meta name="”description”" content="”你的描述内容”">
  <meta name="”keyword”" content="”关键字“">
  <link rel="shortcut icon" href="//x.autoimg.cn/jiage/carprice/pc/img/bulebj32X32.ico">
  <!--<title>车型页</title>-->
  <title>车主价格_null_汽车之家</title>
  <!-- <link rel='stylesheet' href='../sass/index.css'/> -->
  <link rel="stylesheet" href="//s.autoimg.cn/jiage/pc/css/index.css?v=201812231048">
<script type="text/javascript" async="" src="//x.autoimg.cn/bi/mda/ahas_body.min.js?d=18549"></script><script type="text/javascript">
  /*<![CDATA[*/
  (function (M) {
    var N = this.os = {}, L = this.browser = {}, A = M.match(/Web[kK]it[\/]{0,1}([\d.]+)/), _ = M.match(/(Android);?[\s\/]+([\d.]+)?/), B = !!M.match(/\(Macintos

In [48]:
def get_price(js):
    js_header = """var style ={
                    sheet:{
                        insertRule:function(){}
                    }
                }
                var document ={
                    createElement :function(){ return style}
                    ,head:{
                        appendChild:function(){}
                    }
                    ,querySelectorAll :function(){}
                }

                global.window =global
                var window ={
                    getComputedStyle :true
                } 

                obj ={}
                !"""
    js_end = """    
                function $InsertRuleRun$() {
                for ($index$ = 0; $index$ < $rulePosList$.length; $index$++) {
                    var $tempArray$ = $Split$($rulePosList$[$index$], ',');
                    var $temp$ = '';
                    for ($itemIndex$ = 0; $itemIndex$ < $tempArray$.length; $itemIndex$++) {
                        $temp$ += $ChartAt$($tempArray$[$itemIndex$]) + '';
                    }
                    obj["<span class='"+$GetClassName$($index$).split(".")[1]+"'></span>"] =$temp$
                }
            }

        })(document);
            function get_obj(){
            return obj
            }"""

# def get_price(js):
    code1 = js.replace('\n', '')
    code = js_header + code1.replace("})(document)", "") + js_end
    js_code = execjs.compile(code)
    a = js_code.call('get_obj')
    return a

# 车主价格

In [38]:

def get_page(url):
    headers = {
         'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36',
         'Host': 'jiage.autohome.com.cn',
         'Referer': 'https://jiage.autohome.com.cn/',
    }
#     while True:
#         try:
    proxy = get_Proxy()
    print('代理ip: ', proxy)
    proxies = {
        'http': 'http://' + proxy,
        'https': 'https://' + proxy
    }
    res = requests.get(url, headers=headers, proxies=proxies, timeout=5)
    res.encoding = res.apparent_encoding
#         except [ConnectionError, ConnectTimeoutError, TimeoutError]:
#             continue
#         except Exception as e:
#             print(repr(e))
#         else:
    return res
            

In [54]:
from pprint import pprint
import execjs
import pandas as pd

l = []
NUM = 28
rng = list(range(1, NUM))
for i in rng:
# for i in [17]:
    try:
        url = f'https://jiage.autohome.com.cn/price/carlist/p-40624-0-0-0-0-0-{i}-0'
        res = get_page(url)
        soup = BeautifulSoup(res.text)
        print(f'正在处理第 {i} 页...')
        for car in soup.select('ul.car-lists'):
            d = {}
            d['车系'] = soup.select_one('div.athm-sub-nav__car__name').text.strip()
            d['车型'] = car.select_one('p.car-name').text.strip()
            d['购车时间'] = car.select_one('p.bought-time time').text.strip()
            d['购车地点'] = car.select_one('span.bought-location').text.strip()
            d['裸车价'] = car.select_one('span.luochejia-num').text.strip()
            try:
                d['指导价'] = car.select_one('span.text-prompt.green').text.strip()
            except:
                d['指导价'] = None
            d['购车全款'] = car.select_one('span.quankuan-num').text.strip()
#             d['厂商指导价'] = list(get_price(car.select('span.luochejia-num')[1].find('script').text.strip()).values())[0]
            d['车船使用税'] = car.select('span.list-details')[3].text.strip()
            d['厂商指导价'] = None
            d['购置税'] = car.select('span.list-details')[4].text.strip()
            d['交强险'] = car.select('span.list-details')[5].text.strip()
            d['商业险'] = car.select('span.list-details')[6].text.strip()
            d['上牌费'] = car.select('span.list-details')[7].text.strip()
            d['付款方式'] = car.select('span.list-details')[8].text.strip()
            d['促销套餐'] = car.select('span.list-details')[9].text.strip()
            d['砍价技巧'] = car.select('span.list-details')[10].text.strip()
            d['购买商家'] = car.select('span.list-details')[11].text.strip()
        #     pprint(d)
            df = pd.DataFrame(d, index=[0])
            l.append(df)
    except Exception as e:
        print(repr(e))
        print(f'第 {i} 页出错。。。')
        rng.append(i)
        
frame = pd.concat(l, ignore_index=True, sort=False)
frame.to_excel('~/雅阁04.xlsx')
print('ok...')

代理ip:  115.230.61.180:38299
正在处理第 1 页...
代理ip:  115.230.61.180:38299
正在处理第 2 页...
代理ip:  183.163.165.87:42946
正在处理第 3 页...
代理ip:  60.5.125.244:37695
正在处理第 4 页...
代理ip:  60.5.125.244:37695
正在处理第 5 页...
代理ip:  59.55.96.215:44500
正在处理第 6 页...
代理ip:  27.204.39.241:21998
正在处理第 7 页...
代理ip:  27.23.123.191:39237
正在处理第 8 页...
代理ip:  60.31.89.151:35358
正在处理第 9 页...
代理ip:  60.31.89.151:35358
正在处理第 10 页...
代理ip:  115.48.188.30:29550
正在处理第 11 页...
代理ip:  182.86.191.245:21976
正在处理第 12 页...
代理ip:  115.203.99.95:27298
正在处理第 13 页...
代理ip:  220.186.129.191:46748
ProxyError(MaxRetryError("HTTPSConnectionPool(host='jiage.autohome.com.cn', port=443): Max retries exceeded with url: /price/carlist/p-40624-0-0-0-0-0-14-0 (Caused by ProxyError('Cannot connect to proxy.', timeout('select timed out')))"))
第 14 页出错。。。
代理ip:  113.74.52.208:47189
正在处理第 15 页...
代理ip:  122.4.53.69:35058
正在处理第 16 页...
代理ip:  180.109.241.165:47437
正在处理第 17 页...
代理ip:  58.62.115.78:44888
正在处理第 18 页...
代理ip:  58.62.115.78:44888
正在处理第 19

In [34]:
'车船使用税' in res.text

True

In [9]:
soup = BeautifulSoup(html, 'lxml')
soup.select('div.text-con')[0].text.strip().replace('\n', '')

'【行驶过程】刹车力不够大(偏软/刹不住车)'

In [10]:
for i in soup.select('div.text-con'):
    print(i.text.strip().replace('\n', ''))

【行驶过程】刹车力不够大(偏软/刹不住车)
购车2个月后追加口碑                                当前行驶里程 3116 公里当前平均油耗 2.0 升/百公里当前平均耗电量 15.5 千瓦时/百公里【能耗】满电满油的状态，车辆可以行驶500多公里。大都数充满电的时候，车辆显示可以行驶60多公里，偶尔有几次显示超过70了。BMW官方数据，纯电能行驶60公里，经过我自己3000公里的实测，2个成年人，全程空调（23度，3档风） 收音机，能行驶50-55公里之间，不开启空调的话，是可以行驶60公里左右（具体情况可能根据驾驶者的习惯不同，会略有差异）。对于上下班通勤距离50-60公里以内的人来说，日常用车是几乎可以不烧油的。如有稍长距离的行程，油电混用，也能一定程度上节省油耗。 满油满电，车辆显示可以行驶512公里。电池续航：满电，车辆显示可以行驶70公里。曾经有一次显示72公里。但是，实际驾驶中，续航还是50-55之间（开空调）。 高速行驶：高速行驶时，建议开启SAVE模式，让汽油机工作，同时能给电池充电。100-120的时速，纯汽油机行驶，百公里油耗大约5-6升左右。 【保养】目前3000公里多一点，还没进入保养周期。听4S的销售介绍，1.1万公里的时候需要做一次小保养，大约要花费1000元左右。【故障】有一个小问题，但也谈不上是故障。充满电的行驶里程，最少的时候显示不到50公里，最多的时候显示超过70公里。不知道这个会受到什么因素的影响。【吐槽】开了只有2-3个月，目前还没什么要特别吐槽的。自驾游：自驾去过了莫干山，开了30多公里的山路。指向精准，过弯稳健，能应付一些轻度越野的路况，给驾驶者带来很不错的驾驶信心。 汽油发动机：都说三缸机抖动厉害，其实真的还好。怠速的时候，方形盘上立一个硬币还是很轻松的。
【能耗】满电满油的状态，车辆可以行驶500多公里。大都数充满电的时候，车辆显示可以行驶60多公里，偶尔有几次显示超过70了。BMW官方数据，纯电能行驶60公里，经过我自己3000公里的实测，2个成年人，全程空调（23度，3档风） 收音机，能行驶50-55公里之间，不开启空调的话，是可以行驶60公里左右（具体情况可能根据驾驶者的习惯不同，会略有差异）。对于上下班通勤距离50-60公里以内的人来说，日常用车是几乎可以不烧油的。如有稍长距离的行

In [25]:
driver.get('https://www.baidu.com')

In [19]:
from fontTools.ttLib import TTFont

myfont1 = '/home/machao/wKgHFVsUz2-ARgbxAABj-CvJNdg21..ttf'
myfont2 = '/home/machao/wKgHGFsUz2aAXR1iAABj7LvaJg445..ttf'

font1 = TTFont(myfont1)
font2 = TTFont(myfont2)
font1.saveXML('/home/machao/dg21.xml')  # 是  $EDB0
font2.saveXML('/home/machao/g445.xml')  # 是  $EC7E

# 汽车之家 字体反爬

In [44]:
# 汽车之家 
#   字体形状是变化的，多打开几个字体文件发现:
#     同一个字符在不同的字体文件中，xy坐标是变化的，而0，1值和坐标数量是不变的
#     x,y坐标是描绘字体形状的，0，1估计是描述连线的参数
#     猜想：
#         x,y坐标是基于一个标准值作一定范围内的动态变化，这个范围相对于坐标系来说不会太大，如果变化太大则会导致网站字符显示有较大差异。
#         通过几个字体文件计算得出 差值变化在40以内
#     实践：
#         通过比较对象的坐标信息来判断两个不同编码的字符是否是同一个汉字，只要x,y坐标差值在40以内，就认为字形相同，


# 比较函数，用于对比两个字形列表的坐标信息是否相同
#（根据对字体文件的采样调研，认为只要x,y在一定的误差范围内就认为是相同的）

def comp(base_glyf, target_glyf):
    if len(base_glyf) != len(target_glyf):
        return 0
    else:
        mark = 1
        for i in range(len(base_glyf)):
            if abs(base_glyf[i][0] - target_glyf[i][0]) < 40 and abs(base_glyf[i][1] - target_glyf[i][1]) < 40:
                pass
            else:
                mark = 0
                break
        return mark

# 手动确定一组编码和字符的正确对应关系
uni_list = [
    'uniED82', 'uniECCF', 'uniED21', 'uniEC6D', 'uniEDAE', 'uniEE00', 'uniED4C', 'uniED9E', 'uniECEB', 'uniEC37', 'uniEC89', 'uniEDCA', 'uniED16', 
    'uniED68', 'uniECB5', 'uniED07', 'uniEC53', 'uniED94', 'uniEDE6', 'uniED32', 'uniEC7F', 'uniECD1', 'uniEC1D', 'uniEC6F', 'uniEDB0', 'uniECFC', 
    'uniED4E', 'uniEC9B', 'uniEDDB', 'uniEC39', 'uniED7A', 'uniEDCB', 'uniED18', 'uniEC65', 'uniECB6', 'uniEDF7', 'uniED44', 'uniED95'
]
words = '二八六很呢七和高不五得近着了多十远上左好大长短坏是低三一小地九四更矮的右下少'
words_list = list(words)


base_glyf_coordinates = []  # 存储38个字符的坐标信息
for uni in uni_list:
    cdts = font1['glyf'][uni].coordinates
    base_glyf_coordinates.append(list(cdts))
  

target_glyf_coordinates = []
uni_list2 = font2.getGlyphOrder()[1:]
for uni in uni_list2:
    cdts = font2['glyf'][uni].coordinates
    target_glyf_coordinates.append(list(cdts))


x_list=[]
for i, tgc in enumerate(target_glyf_coordinates):
    for j, bgc in enumerate(base_glyf_coordinates):
        if comp(tgc, bgc):
            print(uni_list2[i],words_list[j])
            x_list.append(words_list[j])
            
#分行打印出来，方便和FontCreator中进行比较确认
print(x_list[:16])
print(x_list[16:32])
print(x_list[-6:])


uniEDFD 左
uniED4A 远
uniED9B 近
uniECE8 下
uniEC35 三
uniEC86 好
uniEDC7 呢
uniED14 大
uniED65 不
uniECB2 和
uniED04 地
uniEC50 七
uniED91 小
uniEDE3 短
uniED30 多
uniEC7C 的
uniECCE 得
uniEC1B 四
uniEC6C 矮
uniEDAD 着
uniECFA 八
uniED4B 上
uniEC98 二
uniEDD9 一
uniEC36 五
uniED77 六
uniEDC9 右
uniED15 十
uniEC62 少
uniECB4 长
uniEDF4 九
uniED41 很
uniED93 高
uniECDF 低
uniED31 了
uniEC7E 是
uniEDBE 坏
uniEC1C 更
['左', '远', '近', '下', '三', '好', '呢', '大', '不', '和', '地', '七', '小', '短', '多', '的']
['得', '四', '矮', '着', '八', '上', '二', '一', '五', '六', '右', '十', '少', '长', '九', '很']
['高', '低', '了', '是', '坏', '更']


In [ ]:
&min_behot_time=1562474640&max_behot_time=%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%20%201586864689&max_cursor=17455511579

In [57]:
import requests
import time
import json

headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
    }

url = 'https://www.autohome.com.cn/grade/carhtml/D.html'
res = requests.get(url, headers=headers)
# json.loads(res.text)
"探岳GTE插电混动" in res.text

True

In [31]:
chrome_opts = webdriver.ChromeOptions() 
#         self.chrome_opts.add_argument('--headless')
chrome_opts.add_argument('--no-sandbox')
chrome_opts.add_argument('--disable-gpu')  # 规避bug
chrome_opts.add_experimental_option('excludeSwitches', ['enable-automation'])

driver = webdriver.Chrome(options=chrome_opts)
driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": """
    Object.defineProperty(navigator, 'webdriver', {
      get: () => undefined
    })
  """
})
wait = WebDriverWait(driver, 10)
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36',
}
driver.get('https://www.dcdapp.com/community/4353/koubei')

js_bottom2 = "window.scrollTo(0,document.body.scrollHeight)"
for i in range(4):
    print(i)
    driver.execute_script(js_bottom2)
    time.sleep(1)
time.sleep(3)

0
1
2
3


In [45]:
from pprint import pprint

for i in item[:1]:
    pprint(i['info']["motor_koubei_info"]['structured_info']['extra_info'][1])

{'text': '延安', 'title': '购车地'}


In [13]:
len(item)

2

In [24]:
import pandas as pd
from sqlalchemy import create_engine
import pymongo

In [25]:
client = pymongo.MongoClient(host='192.168.2.149', port=27017)
db = client['newcar_price']
collection = db['autohome_price']
col = db['autohome_price_2020-10-23']

In [26]:
print('autohome_price 数据量：', collection.count())
print('autohome_price_2020 数据量：', col.count())


autohome_price 数据量： 427644
autohome_price_2020 数据量： 4401


<ipython-input-26-cb882ac04173>:1: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  print('autohome_price 数据量：', collection.count())
<ipython-input-26-cb882ac04173>:2: DeprecationWarning: count is deprecated. Use estimated_document_count or count_documents instead. Please note that $where must be replaced by $expr, $near must be replaced by $geoWithin with $center, and $nearSphere must be replaced by $geoWithin with $centerSphere
  print('autohome_price_2020 数据量：', col.count())


In [28]:
data = col.find()
while True:
    try:
        collection.insert_one(data.next())
    except Exception as e:
        print(repr(e))
        break
print('ok...')

StopIteration()
ok...


In [8]:
dir(data)

['_Cursor__address',
 '_Cursor__allow_disk_use',
 '_Cursor__batch_size',
 '_Cursor__check_okay_to_chain',
 '_Cursor__codec_options',
 '_Cursor__collation',
 '_Cursor__collection',
 '_Cursor__collname',
 '_Cursor__comment',
 '_Cursor__data',
 '_Cursor__dbname',
 '_Cursor__die',
 '_Cursor__empty',
 '_Cursor__exhaust',
 '_Cursor__exhaust_mgr',
 '_Cursor__explain',
 '_Cursor__explicit_session',
 '_Cursor__hint',
 '_Cursor__id',
 '_Cursor__killed',
 '_Cursor__limit',
 '_Cursor__manipulate',
 '_Cursor__max',
 '_Cursor__max_await_time_ms',
 '_Cursor__max_scan',
 '_Cursor__max_time_ms',
 '_Cursor__min',
 '_Cursor__modifiers',
 '_Cursor__ordering',
 '_Cursor__projection',
 '_Cursor__query_flags',
 '_Cursor__query_spec',
 '_Cursor__read_concern',
 '_Cursor__read_preference',
 '_Cursor__retrieved',
 '_Cursor__return_key',
 '_Cursor__send_message',
 '_Cursor__session',
 '_Cursor__set_hint',
 '_Cursor__show_record_id',
 '_Cursor__skip',
 '_Cursor__snapshot',
 '_Cursor__spec',
 '__class__',
 '__copy

In [23]:
data.next()

{'_id': ObjectId('5f8db7ca0e7dac922ba7d136'),
 'grabtime': '2020-10-19 15:59:06',
 'website': 'autohome_price_new',
 'prov_ID': 320000,
 'prov_Name': '江苏',
 'city_ID': 321100,
 'city_Name': '镇江',
 'city_Pinyin': 'zhenjiang',
 'brandname': '理念',
 'brandid': '124',
 'factoryname': '广汽本田',
 'factoryid': 32,
 'shopname': '镇江开利广汽本田',
 'shopid': '2111018',
 'tel': '400-872-5083',
 'dealer_address': '镇江市丁卯桥路228号',
 'saleregion': '仅销往江苏省',
 'picurl': 'https://m1.autoimg.cn/cardfs/product/g30/M0A/A6/9E/160x120_0_q87_autohomecar__ChsEoF3ntU6Afg4wAAhqnx7k8Ec611.jpg',
 'vehilename': '雅阁',
 'salespricerange': '169800~259800',
 'guidepricerange': '169800~259800',
 'vehilenameid': 78,
 'salesdesc': '2018款 230TURBO 舒适版 国V',
 'salesdescid': 34198,
 'guideprice': '179800~179800',
 'guidesalesprice': '179800~179800',
 'url': 'https://dealer.autohome.com.cn/handler/other/getdata?__action=dealerlq.getdealerspeclist&dealerId=2111018&seriesId=78',
 'salesprice': 179800,
 'status': 'https://dealer.autohome.co

In [21]:
from pprint import pprint

headers = {
    'User-Agent': 'Dalvik/2.1.0 (Linux; U; Android 7.0; RNE-AL00 Build/HUAWEIRNE-AL00) automobile/4.7.4 cronet/TTNetVersion:c175a3d5 2019-09-05',
    'Content-Type': 'application/x-www-form-urlencoded; charset=UTF-8',
}
for j in range(1, 21):
    url = f'https://www.yinsiduanxin.com/phone-number/page/{j}.html'
    res = requests.get(url, headers=headers)
    res.encoding = res.apparent_encoding

    soup = BeautifulSoup(res.text)
    for i in soup.select('a.clickA'):
        if i.text.split()[0] == '+86':
            print(j, i.text.split()[1], 'https://www.yinsiduanxin.com' + i['href'])

1 17107706354 https://www.yinsiduanxin.com/china-phone-number/verification-code-17107706354.html
1 16532701648 https://www.yinsiduanxin.com/china-phone-number/verification-code-16532701648.html
1 17199741419 https://www.yinsiduanxin.com/china-phone-number/verification-code-17199741419.html
1 17199741418 https://www.yinsiduanxin.com/china-phone-number/verification-code-17199741418.html
2 16532701568 https://www.yinsiduanxin.com/china-phone-number/verification-code-16532701568.html
2 16532701569 https://www.yinsiduanxin.com/china-phone-number/verification-code-16532701569.html
2 17107703029 https://www.yinsiduanxin.com/china-phone-number/verification-code-17107703029.html
3 16532701546 https://www.yinsiduanxin.com/china-phone-number/verification-code-16532701546.html
3 17107706062 https://www.yinsiduanxin.com/china-phone-number/verification-code-17107706062.html
3 16714056626 https://www.yinsiduanxin.com/china-phone-number/verification-code-16714056626.html
3 17107706264 https://www.yins

In [33]:
pprint(res.json()['data']['koubei_list'][0]['sub_list'][0])

{'appended_tag_text': '首次发布',
 'audit_status': 1,
 'card_type': 3,
 'comment_count': 14,
 'comment_list': [{'avatar_url': 'https://p3.pstatp.com/thumb/5d4500031023715f3202',
                   'comment_id': 6871706611841318925,
                   'create_time': '2020-09-13 04:51:45 +0800 CST',
                   'digg_count': 0,
                   'text': '今晚喝多打到一个新能源帕萨特，起步也稳停车也稳加速声音还小滤振也到位，比凯米瑞强太多了，打算周一上班把公车都换成新能源帕萨特，真的舒服',
                   'user_digg': 0,
                   'user_id': 88846264030,
                   'user_name': '周凌宇'},
                  {'avatar_url': 'https://sf1-ttcdn-tos.pstatp.com/img/mosaic-legacy/3793/3131589739~120x256.image',
                   'comment_id': 6838209268127137799,
                   'create_time': '2020-06-14 22:24:25 +0800 CST',
                   'digg_count': 0,
                   'text': '1.4空调怎么样？',
                   'user_digg': 0,
                   'user_id': 1415799353907159,
                   'user_name': '用户3612077647522'}],
 'c

## 下载音乐

In [4]:
import requests

def get_music(url, name='music'):
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36'
        }
    with open(f'/home/machao/音乐/music/{name}.mp3', 'wb') as f:
        f.write(requests.get(url, headers=headers).content)
    print('ok...')

In [7]:
url = 'https://webfs.yun.kugou.com/202010220932/683a439f7c075448386c976e07c586fe/G175/M02/04/07/T4cBAF38rOWAbLu0AEaiShiMjRA434.mp3'
name = '岑雨桥-九点半(儋州调声正式版)'
get_music(url, name)

ok...


In [ ]:
from requests_html import HTMLSession
import requests
import json

In [ ]:
url = 'https://jia.chexiang.com/store/queryStoreByXY.htm'
session = requests.Session()
headers = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Connection': 'keep-alive',
    'Content-Type': 'application/json',
    'Cookie': 'bbbbbbbbbbbbbbb=FDLAFDCMHGFOOLLHNHNCKLFDJDIIKHFCOGNGBDDBJCEDFCEOMHFICOOCLGFAEOBKCFODFJFGKOAEILMMFBGKCPKENCLHIOAPHCBGDGENNMJNHJKLCDECKKJLHCDGGFHA; scookie0=3c62e570a7ea4e93bab9dbd625e9ac7d; wemall_opentype_c=10; wemall_userkey_c=1593b689-003b-484c-a1d1-9028f158ba0d; user_trace_cookie=CiCcOF8eM1mz8VSkFSVvAg==; cityId=310100; city.id=310100; city.name=%E4%B8%8A%E6%B5%B7; city_id=310100; city_name=%E4%B8%8A%E6%B5%B7; area.code=244000; city.domain=.chexiang.com; wcid=1869C0D8-735D-4F2B-B472-3564E58BD03B; Hm_lvt_f8ba65b380f665911ecab4df90bfe056=1595814797; Hm_lpvt_f8ba65b380f665911ecab4df90bfe056=1595814797; bbbbbbbbbbbbbbb=IBKICBNMIICBNNBFHPPBGBNPEIDJBHMCCPFIDMEPADEDJCBKFJJLJCIDHEPABFFMADEDDCAKHNKEDJPHFDNHEFNEBBMHEEJLHHFEBLCHAIIKHFKBLHGNOMOBMKMKICIE; scookie1=1597116652145; scookie2=1597117854403',
    'Host': 'jia.chexiang.com',
    'Origin': 'https://jia.chexiang.com',
    'Referer': 'https://jia.chexiang.com/t/stores.htm',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36',
    'X-Requested-With': 'XMLHttpRequest',
}
payload = {
    "D": "a", "X": 31.28697, "Y": 121.50253, "areaCode": "310100"
}

res = requests.post(url, headers=headers, data=json.dumps(payload))
print(res.status_code)

In [ ]:
import pymongo
import pandas as pd

In [ ]:
mongo_client = pymongo.MongoClient(host='192.168.2.149', port=27017)
db = mongo_client['shop_name']
collection = db['chexiang']

In [ ]:
res.encoding = res.apparent_encoding
data = json.loads(res.text)
l = []

for dat in data['obj']:
    text = {}
    text['storeName'] = dat['storeName']
    text['address'] = dat['address']
    text['telephone'] = dat['telephone']
    text['mobile'] = dat['mobile']
    text['info'] = {
        'storeId': dat['storeId'],
        'storeNo': dat['storeNo'],
        'intro': dat['intro'],
        'businesstime': dat['businesstime'],
        'localX': dat['localX'],
        'localY': dat['localY'],
        'distanceFromMe': dat['distanceFromMe'],
        'image1': dat['image1'],
        'image2': dat['image2'],
        'image3': dat['image3'],
        'image4': dat['image4'],
        'baiduCityCode': dat['baiduCityCode'],
        'areaCode': dat['areaCode'],
        'areaName': dat['areaName'],
        'provinceId': dat['provinceId'],
        'provinceName': dat['provinceName'],
        'cityId': dat['cityId'],
        'cityName': dat['cityName'],
        'distId': dat['distId'],
        'distName': dat['distName'],
        'storeType': dat['storeType'],
        'storeScore': dat['storeScore'],    
    }
    l.append(text)

result = collection.insert_many(l)
result

In [ ]:
r = requests.get('http://www.baidu.com')
r.headers

In [ ]:
import logging

logging.basicConfig(level=logging.INFO,
                    format='%(levelname)s [%(filename)s:%(lineno)d]'
                           ':  %(message)s'
                           ' - %(asctime)s', datefmt='[%d/%b/%Y %H:%M:%S]',
                    )    
logger = logging.getLogger(__name__)

if __name__ == "__main__":
        logger.info("this is a info log")
        logger.info("this is a info log1")

In [2]:
url = 'http://www.niuniuqiche.com/v2/sell_cars?brand_name=%E8%B7%AF%E8%99%8E&page=100'
headers = {
    'Accept': '*/*',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Connection': 'keep-alive',
#     'Content-Type': 'application/json',
    'Cookie': 'Hm_lvt_9e0791875b44c6cb9634aaf00c1d4986=1600932519; Hm_lvt_9e884c78bd7bfb5c8bee0f9c00fa772f=1600932519; _bl_uid=eUkbLfmjg51hm4vd4x27a90g7q1X; Hm_lpvt_9e0791875b44c6cb9634aaf00c1d4986=1600934918; Hm_lpvt_9e884c78bd7bfb5c8bee0f9c00fa772f=1600934918; _niu_niu_session=Y2lBU2NtREZ6ckRoYkJnSWJCV1U0V08xcmUwUC9wcGFCSGZZMDBNQURqNXg4WGp5ZnhmWG02YTh2Tm5tZ2ViY3ljZmNLOFFQOVRndGhlaGZrWVROSXFneVRxejIwbndkUUJVZHViT0V6ZjVPSDNWQzMvVHRMVytIcFBqMDc0MVl5cENqaHdLdWhwSk5jZWJER3dUNGdCT1pyQWx4UndQNStGVzY3Rjg3VTg1YjByaER2Yys5U3hlSFQ3Mk1DbjMrZWpqcGQ4SDlBS2NJSEEyMkhHeVJUbW5uNFg2RGFyY2NjaDdjWm1tWlVSRTFsNW1rZTgzdzdtRnp1bDVrZFRVdS0tTHUwWDRlUW51MGtGbGV6KzBSZzdSdz09--b32deacded7d6a98c85b1a8882c888aff5f112f6; SERVERID=8a9626b5940df00f9a9360e60b5c790c|1600936038|1600936038',
#     'Host': 'jia.chexiang.com',
#     'Origin': 'https://jia.chexiang.com',
#     'Referer': 'https://jia.chexiang.com/t/stores.htm',
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36',
}
try:
    res = requests.get(url, headers=headers)
    res.encoding = 'utf-8'
except Exception as e:
    print(repr(e))

res.text

'Forbidden\n'

In [ ]:
from requests.utils.dict_from_cookiejar

In [67]:
s = 'Hm_lvt_9e0791875b44c6cb9634aaf00c1d4986=1600932519; Hm_lvt_9e884c78bd7bfb5c8bee0f9c00fa772f=1600932519; _bl_uid=eUkbLfmjg51hm4vd4x27a90g7q1X; Hm_lpvt_9e0791875b44c6cb9634aaf00c1d4986=1600934918; Hm_lpvt_9e884c78bd7bfb5c8bee0f9c00fa772f=1600934918; _niu_niu_session=Y2lBU2NtREZ6ckRoYkJnSWJCV1U0V08xcmUwUC9wcGFCSGZZMDBNQURqNXg4WGp5ZnhmWG02YTh2Tm5tZ2ViY3ljZmNLOFFQOVRndGhlaGZrWVROSXFneVRxejIwbndkUUJVZHViT0V6ZjVPSDNWQzMvVHRMVytIcFBqMDc0MVl5cENqaHdLdWhwSk5jZWJER3dUNGdCT1pyQWx4UndQNStGVzY3Rjg3VTg1YjByaER2Yys5U3hlSFQ3Mk1DbjMrZWpqcGQ4SDlBS2NJSEEyMkhHeVJUbW5uNFg2RGFyY2NjaDdjWm1tWlVSRTFsNW1rZTgzdzdtRnp1bDVrZFRVdS0tTHUwWDRlUW51MGtGbGV6KzBSZzdSdz09--b32deacded7d6a98c85b1a8882c888aff5f112f6; SERVERID=8a9626b5940df00f9a9360e60b5c790c|1600936038|1600936038'

In [68]:
d = {}
for i in s.split('; '):
    d[i.split('=')[0]] = i.split('=')[1]
print(d)

{'Hm_lvt_9e0791875b44c6cb9634aaf00c1d4986': '1600932519', 'Hm_lvt_9e884c78bd7bfb5c8bee0f9c00fa772f': '1600932519', '_bl_uid': 'eUkbLfmjg51hm4vd4x27a90g7q1X', 'Hm_lpvt_9e0791875b44c6cb9634aaf00c1d4986': '1600934918', 'Hm_lpvt_9e884c78bd7bfb5c8bee0f9c00fa772f': '1600934918', '_niu_niu_session': 'Y2lBU2NtREZ6ckRoYkJnSWJCV1U0V08xcmUwUC9wcGFCSGZZMDBNQURqNXg4WGp5ZnhmWG02YTh2Tm5tZ2ViY3ljZmNLOFFQOVRndGhlaGZrWVROSXFneVRxejIwbndkUUJVZHViT0V6ZjVPSDNWQzMvVHRMVytIcFBqMDc0MVl5cENqaHdLdWhwSk5jZWJER3dUNGdCT1pyQWx4UndQNStGVzY3Rjg3VTg1YjByaER2Yys5U3hlSFQ3Mk1DbjMrZWpqcGQ4SDlBS2NJSEEyMkhHeVJUbW5uNFg2RGFyY2NjaDdjWm1tWlVSRTFsNW1rZTgzdzdtRnp1bDVrZFRVdS0tTHUwWDRlUW51MGtGbGV6KzBSZzdSdz09--b32deacded7d6a98c85b1a8882c888aff5f112f6', 'SERVERID': '8a9626b5940df00f9a9360e60b5c790c|1600936038|1600936038'}


In [72]:
import six
cookie_list = [{'name': k, 'value': v} for k, v in six.iteritems(d)]
for i in cookie_list:
    print(i)

{'name': 'Hm_lvt_9e0791875b44c6cb9634aaf00c1d4986', 'value': '1600932519'}
{'name': 'Hm_lvt_9e884c78bd7bfb5c8bee0f9c00fa772f', 'value': '1600932519'}
{'name': '_bl_uid', 'value': 'eUkbLfmjg51hm4vd4x27a90g7q1X'}
{'name': 'Hm_lpvt_9e0791875b44c6cb9634aaf00c1d4986', 'value': '1600934918'}
{'name': 'Hm_lpvt_9e884c78bd7bfb5c8bee0f9c00fa772f', 'value': '1600934918'}
{'name': '_niu_niu_session', 'value': 'Y2lBU2NtREZ6ckRoYkJnSWJCV1U0V08xcmUwUC9wcGFCSGZZMDBNQURqNXg4WGp5ZnhmWG02YTh2Tm5tZ2ViY3ljZmNLOFFQOVRndGhlaGZrWVROSXFneVRxejIwbndkUUJVZHViT0V6ZjVPSDNWQzMvVHRMVytIcFBqMDc0MVl5cENqaHdLdWhwSk5jZWJER3dUNGdCT1pyQWx4UndQNStGVzY3Rjg3VTg1YjByaER2Yys5U3hlSFQ3Mk1DbjMrZWpqcGQ4SDlBS2NJSEEyMkhHeVJUbW5uNFg2RGFyY2NjaDdjWm1tWlVSRTFsNW1rZTgzdzdtRnp1bDVrZFRVdS0tTHUwWDRlUW51MGtGbGV6KzBSZzdSdz09--b32deacded7d6a98c85b1a8882c888aff5f112f6'}
{'name': 'SERVERID', 'value': '8a9626b5940df00f9a9360e60b5c790c|1600936038|1600936038'}


## 接码平台

In [20]:
from requests_html import HTMLSession
import execjs
from pprint import pprint
from tqdm import tqdm


session = HTMLSession()
headers = {
    'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.89 Safari/537.36',
}

phones = []
for i in tqdm(range(1, 6)):
    url = f'https://mianfeijiema.com/China-Phone-Number/list_{i}.html'
#     proxy = get_Proxy()
#     print('use proxy: ', proxy)
#     proxies = {
#       "http": f"http://{proxy}",
#       "https": f"http://{proxy}",
#     }
    try:
        res = session.get(url, headers=headers)
    #     res.html.render()
        res.encoding = 'utf-8'
    except Exception as e:
        print(repr(e))
    
    phone_list = res.html.find('h3.number.black-text.listimg span')
    for phone in phone_list:
        phones.append(phone.text.split(' ')[1])

with open('/home/machao/公共的/田泽瑞/田泽瑞--交接/牛牛汽车/phones', 'w') as f:
    for p in phones:
        f.write(p + ' ' + 'https://mianfeijiema.com/sms/86' + p + '\n')
        
print('ok...')

100%|██████████| 5/5 [00:01<00:00,  2.97it/s]

ok...


#  牛牛注册

In [3]:
from requests_html import HTMLSession
from selenium import webdriver
import time


class GetMessage(object):
    def __init__(self):
        self.session = HTMLSession()
        self.headers = {
            'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/85.0.4183.102 Safari/537.36',
        }
        self.dct = {}

    def get_number(self, url):
        try:
            res = self.session.get(url, headers=self.headers)
            assert res.status_code == 200
            res.encoding = res.apparent_encoding
        except Exception as e:
            print('!!请求号码页面出现异常：', repr(e))
        else:
            phones = res.html.find('div.col-md-4 p.number')
            href = res.html.find('div.col-md-4 a', containing='读取短信')
            with open('/home/machao/公共的/田泽瑞/田泽瑞--交接/牛牛汽车/phones', 'a') as f:
                for p, h in zip(phones, href[::-1]):
                    # print(p.text, h.text, url + h.attrs['href'])
                    self.dct[str(p.text)] = url + h.attrs['href']
                    f.write(p.text+ ' ' + url + h.attrs['href'] + '\n')
        return self.dct

    def get_data(self, msg_url):
        while True:
            try:
                res = self.session.get(msg_url, headers=self.headers)
                assert res.status_code == 200
                res.encoding = res.apparent_encoding
                time.sleep(10)
            except Exception as e:
                print('!!请求短信页面出现异常：', repr(e))
            else:
                try:
                    ma = res.html.search('【牛牛汽车】您的验证码是{},该验证码将在15分钟后失效')[0]
                except IndexError:
                    time.sleep(5)
                else:
                    break

        return ma            
        


# if __name__ == "__main__":
#     msg = GetMessage()
#     url = 'http://www.xnsms.com'
#     msg.get_number(url)
    # print(get_data('https://www.materialtools.com/SMSContent/147'))

    # print(get_msg.dct)

In [29]:
from redis import Redis
from io import BytesIO
import json

In [40]:
from urllib.parse import unquote
from pprint import pprint
from PIL import Image
from selenium.common.exceptions import NoSuchElementException


# 注册
BORDER = 6 

class CrackGeetest():
    def __init__(self, phone):
        self.headers = {
            'Accept': '*/*',
            'Accept-Encoding': 'gzip, deflate',
            'Accept-Language': 'zh-CN,zh;q=0.9',
            'Connection': 'keep-alive',
#             'Host': 'api.geetest.com',
#             'Referer': 'http://www.niuniuqiche.com/',
            'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/86.0.4240.75 Safari/537.36',
        }
        chrome_options = Options()
        # chrome_options.add_argument('--headless')
        chrome_options.add_argument('--proxy-server=' + CrackGeetest.get_Proxy())
        chrome_options.add_experimental_option('excludeSwitches', ['enable-automation'])
        self.url = 'http://www.niuniuqiche.com/login'
        self.browser = webdriver.Chrome(options=chrome_options)

        self.browser.maximize_window()
        self.wait = WebDriverWait(self.browser, 20)
        self.phone = phone
#         self.redis_cli = Redis(host="192.168.1.248", port=6379, db=3)
#         self.fa_cookie = open("cookie0", "a")

#         self.cookie = {}

    @classmethod
    def get_Proxy(cls):
        url = 'http://192.168.2.120:5000'
        headers = {
            'Connection': 'close',
        }
        proxy = requests.get(url, headers=headers, auth=('admin', 'zd123456'), timeout=30).text[0:-6]
        return proxy

    # 程序完成，自动结束程序
    def __del__(self):
        self.browser.close()
#         self.fa_cookie.close()

    def get_vcode(self, msg_url):
        while True:
            try:
                res = requests.get(msg_url, headers=self.headers)
                res.encoding = res.apparent_encoding
                ma = re.findall(r'【牛牛汽车】您的验证码是(\d{4}),该验证码将在15分钟后失效', res.text)[0]
            except IndexError:
                continue
            except Exception as e:
                print(repr(e))
                break
            else:
                return ma

    def open(self):
        """
        打开网页
        :return: None
        """
        self.browser.get(self.url)
        self.wait.until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="wrapper"]/div[1]/div/div/div[2]/ul/li[2]/a'))
        )
        self.browser.find_element_by_xpath("//ul/li[2]/a").click()
        time.sleep(0.3)
        self.browser.find_element_by_xpath("//div[@id='modal_sign_up']//input[@id='user_mobile']").send_keys(self.phone)
        time.sleep(0.8)
        self.browser.find_element_by_xpath('//div[@id="modal_sign_up"]//div[@class="geetest_radar_tip"]').click()
        time.sleep(5)
        # 保存图片
        self.browser.find_element_by_xpath('//div[@class="geetest_fullpage_click_wrap"]//canvas').screenshot('/home/machao/geetest.png')
        js = "document.getElementsByClassName('geetest_canvas_fullbg geetest_fade geetest_absolute')[0].style=\'null\'"
        self.browser.execute_script(js)
        time.sleep(3)
        self.browser.find_element_by_xpath('//div[@class="geetest_widget"]//canvas').screenshot('/home/machao/fullimg.png')
        
    def is_pixel_equal(self, img1, img2, x, y):
        """
        判断两个像素是否相同
        :param image1: 图片1
        :param image2: 图片2
        :param x: 位置x
        :param y: 位置y
        :return: 像素是否相同
        """
        # 取两个图片的像素点
        pixel1 = img1.load()[x, y]
        pixel2 = img2.load()[x, y]
        threshold = 60
        if abs(pixel1[0] - pixel2[0]) < threshold and abs(pixel1[1] - pixel2[1]) < threshold and abs(
                pixel1[2] - pixel2[2]) < threshold:
            return True
        else:
            return False

    def get_offset(self, img1, img2):
        """
        获取缺口偏移量
        img1: 缺口图
        img2: 完整图
        """
        left = 60
#         print(img1.size[0])
#         print(img1.size[1])
        for i in range(left, img1.size[0]):
            for j in range(img1.size[1]):
                if not self.is_pixel_equal(img1, img2, i, j):
                    left = i
                    return left + 1
        return left


    def move_to_gap(self, offset):
        btn = self.browser.find_element_by_class_name('geetest_slider_button')
        mouse_action = ActionChains(self.browser).click_and_hold(btn)
        # 因此要将滑块移动至指定位置，最多需要执行move_steps步
        move_steps = int(offset/4)
        for i in range(0, move_steps):
            # 路程前半部分速度较快
            if i < int(move_steps/2):
                # sleep(random.randint(1, 10) / 500)#
                # 滑块每次向右移动四个像素，鼠标Y坐标在上下5个像素内随机摆动
                mouse_action.move_by_offset(4, random.randint(-5, 5)).perform()
            else:
                # 在路程的后半段，越接近终点速度越慢
                # 每次移动之前sleep一段时间，时间为总距离与已移动距离方差的倒数
                seed = 90.0/(pow(move_steps, 2) - pow(i, 2))
#                 time.sleep(seed)
                mouse_action.move_by_offset(4, random.randint(-5, 5)).perform()

            mouse_action = ActionChains(self.browser).click_and_hold(btn)
        # 到达终点时，左右摆动，假装做调整。
        #sleep(0.1)
        #mouse_action.move_by_offset(5, random.randint(-5, 5)).perform()
        #sleep(0.2)
        #mouse_action.move_by_offset(-4, random.randint(4,5)).perform()
        time.sleep(0.1)
        # 松开鼠标
        mouse_action.release().perform()
        time.sleep(1)
        if '验证成功' in self.browser.page_source:
            print('验证成功')
            return 1
        else:
            print('再接再厉吧!')
            return 0
        
#         # 获取验证码
#         self.browser.find_element_by_xpath('//*[@id="btn_valid_code"]').click()
#         ma = self.get_vcode('http://www.xnsms.com/test/getPhoneData?phone=17087330261')
#         self.browser.find_element_by_id('valid_code').send_keys(ma)
#         time.sleep(0.3)
#         self.browser.find_element_by_id('sign_up_next').click()

def main():
    while True:
        try:
            crack = CrackGeetest('17043769770')
            crack.open()
            img1 = Image.open('/home/machao/geetest.png')
            img2 = Image.open('/home/machao/fullimg.png')
            # 获取偏移量
            offset = crack.get_offset(img1, img2)
            print('偏移量：', offset)
            code = crack.move_to_gap(offset)
            if code == 1:
                break
        except NoSuchElementException:
            continue

        
        
if __name__ == '__main__':
    main()

        


偏移量： 82
验证成功


In [51]:
driver=webdriver.Chrome()
driver.maximize_window()
driver.get("https://www.csdn.net/")

In [ ]:
对滑块速度有校验的：
    1、直接将滑块瞬间拖至缺口处；（即selenium不对滑块移动速度做任何处理）
    2、将滑块匀速拖至缺口处；（包括先匀加速后匀减速）
以上两类行为有明显的程序操作特征，而人类操作滑块有以下特征：
    1、滑块速度先快后慢，但加速度不恒定；
    2、在接近缺口的时候，滑块可能会左右摇摆来适应缺口位置；
    3、滑动过程中，鼠标的Y坐标一般会有上下浮动的情况

In [4]:
chrome_options = Options()
proxy = get_Proxy()
chrome_options.add_argument('--proxy-server=' + proxy)
chrome_options.add_experimental_option('excludeSwitches', ['enable-automation'])

url = 'http://www.niuniuqiche.com/login'
driver = webdriver.Chrome(options=chrome_options)
driver.maximize_window()
driver.get(url)
# time.sleep(2)
# driver.close()

In [105]:
d = {}

In [106]:
for c in cookies:
    d[c['name']] = c['value']
d

{'_bl_uid': 'R3k2sfg9mdtaX47shwOFj7j4ahX0',
 '_niu_niu_session': 'MVN0L2RUaFVSTlArR2Q1TldZcnJvU2ttdXJpcjI4bGJXN1c5WWJRYW1ITWh0UGRuS1NLMXBEelJ2VlBreHJpVVFDVEowcWlnTEkxTS9PMVROOGpMUnBjRi9nTm5SK01QdzVjaklEWkphbkhJSkRzbjRZQklEOGd2R2dCZndoeFJQcnlMdjk2L1lJWE9QTkZ5QTRDbFB3PT0tLVRxVmRKRFpSMzNzVklTTW9ha1ZoL1E9PQ%3D%3D--aa29d3632f82b79ce9e4306235618f16522cf7f2',
 'SERVERID': 'dbf759c49f3646fe8639abee23e017e8|1601282437|1601282426',
 'Hm_lpvt_9e0791875b44c6cb9634aaf00c1d4986': '1601282434',
 'Hm_lpvt_9e884c78bd7bfb5c8bee0f9c00fa772f': '1601282434',
 'Hm_lvt_9e0791875b44c6cb9634aaf00c1d4986': '1601282434',
 'Hm_lvt_9e884c78bd7bfb5c8bee0f9c00fa772f': '1601282434'}

In [110]:
import json

In [112]:
s = json.dumps(d)
print(type(s))
s

<class 'str'>


'{"_bl_uid": "R3k2sfg9mdtaX47shwOFj7j4ahX0", "_niu_niu_session": "MVN0L2RUaFVSTlArR2Q1TldZcnJvU2ttdXJpcjI4bGJXN1c5WWJRYW1ITWh0UGRuS1NLMXBEelJ2VlBreHJpVVFDVEowcWlnTEkxTS9PMVROOGpMUnBjRi9nTm5SK01QdzVjaklEWkphbkhJSkRzbjRZQklEOGd2R2dCZndoeFJQcnlMdjk2L1lJWE9QTkZ5QTRDbFB3PT0tLVRxVmRKRFpSMzNzVklTTW9ha1ZoL1E9PQ%3D%3D--aa29d3632f82b79ce9e4306235618f16522cf7f2", "SERVERID": "dbf759c49f3646fe8639abee23e017e8|1601282437|1601282426", "Hm_lpvt_9e0791875b44c6cb9634aaf00c1d4986": "1601282434", "Hm_lpvt_9e884c78bd7bfb5c8bee0f9c00fa772f": "1601282434", "Hm_lvt_9e0791875b44c6cb9634aaf00c1d4986": "1601282434", "Hm_lvt_9e884c78bd7bfb5c8bee0f9c00fa772f": "1601282434"}'

In [114]:
type(json.loads(s))

dict

In [ ]:
# js 用户代理字符串检测脚本
js_code = '''
    var client = function(){
    // 呈现引擎
    var engine = {
        ie: 0,
        gecko: 0,
        webkit: 0,
        khtml: 0,
        opera: 0,
        // 完整的版本号
        ver: null
    };
    // 浏览器
    var browser = {
        // 主要浏览器
        ie: 0,
        firefox: 0,
        safari: 0,
        konq: 0,
        opera: 0,
        chrome: 0,
        // 具体版本号
        ver: null
    };
    // 平台、设备和操作系统
    var system = {
        win: false,
        mac: false,
        xll: false,
        // 移动设备
        iphone: false,
        ipod: false,
        ipad: false,
        ios: false,
        android: false,
        nokiaN: false,
        winMobile: false,
        // 游戏系统
        wii: false,
        ps: false
    };
    // 检测呈现引擎和浏览器
    var ua = navigator.userAgent;
    if(window.opera){
        engine.ver = browser.ver = window.opera.version();
        engine.opera = browser.opera = parseFloat(engine.ver);
    }else if(/AppleWebKit\/(\S+)/.test(ua)){
        engine.ver = RegExp["$1"];
        engine.webkit = parseFloat(engine.ver);
        // 确定是 Chrome 还是 Safari
        if(/Chrome\/(\S+)/.test(ua)){
            browser.ver = RegExp["$1"];
            browser.chrome = parseFloat(browser.ver);
        }else if(/Version\/(\S+)/.test(ua)){
            browser.ver = RegExp("$1");
            browser.safari = parseFloat(browser.ver);
        }else{
            // 近似地确定版本号
            var safariVersion = 1;
            if(engine.webkit < 100){
                safariVersion = 1;
            }else if(engine.webkit < 312){
                safariVersion = 1.2;
            }else if(engine.webkit < 412){
                safariVersion = 1.3;
            }else{
                safariVersion = 2;
            }
            browser.safari = browser.ver = safariVersion;
        }
    }else if(/KHTML\/(\S+)/.test(ua) || /Konqueror\/([^;]+)/.test(ua)){
        engine.ver = browser.ver = RegExp["$1"];
        engine.khtml = browser.konq = parseFloat(engine.ver);
    }else if(/rv:([^\)]+)\) Gecko\/\d{8}/.test(ua)){
        engine.ver = RegExp["$1"];
        engine.gecko = parseFloat(engine.ver);
        // 确定是不是 Firefox
        if(/Firefox\/(\S+)/.test(ua)){
            browser.ver = RegExp["$1"];
            browser.firefox = parseFloat(browser.ver);
        }
    }else if(/MSIE ([^;]+)/.test(ua)){
        engine.ver = browser.ver = RegExp["$1"];
        engine.ie = browser.ie = parseFloat(browser.ver);
    }
    // 检测浏览器
    browser.ie = engine.ie;
    browser.opera = engine.opera;
    // 检测平台
    var p = navigator.platform;
    system.win = p.indexOf("Win") == 0;
    system.mac = p.indexOf("Mac") == 0;
    system.xll = (p == "Xll") || (p.indexOf("Linux") == 0);
    // 检测 Windows 操作系统
    if(system.win){
        if(/Win(?:dows)?([^do]{2})\s?(\d+\.\d+)?/.test(ua)){
            if(RegExp["$1"] == "NT"){
                switch (RegExp["$2"]) {
                    case "5.0":
                        system.win = "2000";
                        break;
                    case "5.1":
                        system.win = "XP";
                        break;
                    case "6.0":
                        system.win = "Vista";
                        break;
                    case "6.1":
                        system.win = "7";
                        break;
                    default:
                        system.win = "NT";
                        break;
                }
            }else if(RegExp["$1"] == "9x"){
                system.win = "ME";
            }else{
                system.win = RegExp["$1"];
            }
        }
    }
    // 移动设备
    system.iphone = ua.indexOf("iPhone") > -1;
    system.ipod = ua.indexOf("iPod") > -1;
    system.ipad = ua.indexOf("iPad") > -1;
    system.nokiaN = ua.indexOf("NokiaN") > -1;
    // Windows mobile
    if(system.win == "CE"){
        system.winMobile = system.win;
    }else if(system.win == "Ph"){
        if(/Windows Phone OS (\d+.\d+)/.test(ua)){
            system.win = "Phone";
            system.winMobile = parseFloat(RegExp["$1"]);
        }
    }
    // 检测 iOS 版本
    if(system.mac && ua.indexOf("Mobile") > -1){
        if(/CPU (?:iPhone )?OS (\d+_\d+)/.test(ua)){
            system.ios = parseFloat(RegExp.$1.replace("_", "."));
        }else{
            system.iOS = 2;// 不能真正检测出来，所以只能猜测
        }
    }
    // 检测 Android 版本
    if(/Android (\d+\.\d+)/.test(ua)){
        system.android = parseFloat(RegExp.$1);
    }
    // 游戏系统
    system.wii = ua.indexOf("Wii") > -1;
    system.ps = /playstation/i.test(ua);
    // 返回这些对象
    return {
        engine: engine,
        browser: browser,
        system: system
    };
}();
'''